In [2]:
import pandas as pd
import numpy as np
import codecs
from scipy.io import loadmat
import glob
import csv
import html
import matplotlib.pyplot as plt
import urllib
%matplotlib inline  
import random 
random.seed(99)

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000


# To reproduce load model and predict on wh_both dev set:

source ~/.bash rc
tensorflow
cd ~/Repos/attention_attention/mltagger/exp_scripts/
sudo ./exp_load.sh  bl /Users/Maria/Repos/attention_attention/mltagger/exp/testoutput/test-fce-bl-32-1.0-0.4-1.0-66-0.25 
./exp_load.sh Mean_fix_dur /Users/Maria/Repos/attention_attention/mltagger/exp/testoutput/test-fce-Mean_fix_dur-32-0.1-0.2-1.0-66-1.0
/Users/Maria/Repos/attention_attention/mltagger/exp/testoutput/test-semeval_neg_sent-bl-32-1.0-0.4-1.0-66-0.25
/Users/Maria/Repos/attention_attention/mltagger/exp/testoutput/test-semeval_neg_sent-Mean_fix_dur-32-1.0-0.8-1.0-66-decreasing
/Users/Maria/Repos/attention_attention/mltagger/exp/testoutput/test-semeval_pos_sent-bl-32-0.1-0.4-1.0-66-decreasing
./exp_load.sh Mean_fix_dur /Users/Maria/Repos/attention_attention/mltagger/exp/testoutput/test-semeval_pos_sent-Mean_fix_dur-32-0.1-0.4-1.0-66-0.125

./exp_load.sh bl /Users/Maria/Repos/attention_attention/mltagger/exp/testoutput/test-wh_both-bl-32-0.1-0.2-1.0-66-0.125
./exp_load.sh Mean_fix_dur /Users/Maria/Repos/attention_attention/mltagger/exp/testoutput/test-wh_both-Mean_fix_dur-32-0.1-0.6-1.0-66-decreasing

In [4]:
old_foldernames = ["aws5/wh_both_bl-32-0.1-1.0-1.0-66-decreasing/",
                "aws5/wh_both-Mean_fix_dur-32-0.1-1.0-1.0-66-decreasing/",
               "aws3/semeval_neg_sent_bl-32-1.0-0.4-1.0-66-0.25/",
               "aws3/semeval_neg_sent-Mean_fix_dur-32-1.0-0.6-1.0-66-0.5/",
               "aws2/fce_bl-32-1.0-0.6-1.0-66-0.25/",
               "aws2/fce-Mean_fix_dur-32-1.0-1.0-1.0-66-0.25/"
              ]

foldernames = ["testoutput/test-fce-bl-32-1.0-0.4-1.0-66-0.25/",
               "testoutput/test-fce-Mean_fix_dur-32-0.1-0.2-1.0-66-1.0/",
               "testoutput/test-semeval_neg_sent-bl-32-1.0-0.4-1.0-66-0.25/",
               "testoutput/test-semeval_neg_sent-Mean_fix_dur-32-1.0-0.8-1.0-66-decreasing/",
               #"testoutput/test-semeval_pos_sent-bl-32-0.1-0.4-1.0-66-decreasing/",
               #"testoutput/test-semeval_pos_sent-Mean_fix_dur-32-0.1-0.4-1.0-66-0.125/",
               "testoutput/test-wh_both-bl-32-0.1-0.2-1.0-66-0.125/",
               "testoutput/test-wh_both-Mean_fix_dur-32-0.1-0.6-1.0-66-decreasing/"
]
domain = "wh_both"
alldoms = pd.DataFrame()
for folder in foldernames:
    print(folder)
    dom = '-'.join([folder[11:].split('-')[2],folder[11:].split('-')[1]])
    allsents = pd.DataFrame()
    folderpath = "mltagger/exp/"+folder
    outputfile = folderpath+"modelload_predictions_dv_"+domain+"/output.txt"
    filelist = [li.split('\n') for li in open(outputfile).read().split('\n\n')]
    wordsplit = [[w.split('\t') for w in sent] for sent in filelist]
    
    for sent in wordsplit[:50]:
        if sent[0][0] == '1': #if gold label is 1:
            #print(sent[0])
            onesent = pd.DataFrame.from_records(sent[1:], columns=['Word', 'Label', 'Pred_gaze', 'Attention']) #not keeping the label rows
            onesent = onesent[['Word', 'Attention']]
            onesent.Attention = onesent.Attention.astype(float)
            mean = onesent.Attention.mean()
            onesent.loc[onesent.Attention > mean, 'Word'] = onesent[onesent.Attention > mean]["Word"].apply(lambda x: "\textbf{"+str(x)+"}")
            allsents = pd.concat([allsents, onesent], axis=0)
    allsents.columns = [dom , 'attention']
    allsents.reset_index(drop=True, inplace=True)

    alldoms = pd.concat([alldoms, allsents[[dom]]], axis=1)


testoutput/test-fce-bl-32-1.0-0.4-1.0-66-0.25/
testoutput/test-fce-Mean_fix_dur-32-0.1-0.2-1.0-66-1.0/
testoutput/test-semeval_neg_sent-bl-32-1.0-0.4-1.0-66-0.25/
testoutput/test-semeval_neg_sent-Mean_fix_dur-32-1.0-0.8-1.0-66-decreasing/
testoutput/test-wh_both-bl-32-0.1-0.2-1.0-66-0.125/
testoutput/test-wh_both-Mean_fix_dur-32-0.1-0.6-1.0-66-decreasing/


In [ ]:
alldoms[7000:7500]

In [5]:
alldoms[180:202]

,bl-fce,Mean_fix_dur-fce,bl-semeval_neg_sent,Mean_fix_dur-semeval_neg_sent,bl-wh_both,Mean_fix_dur-wh_both
180,@CharlesClassiqk:,\textbf{@CharlesClassiqk:},\textbf{@CharlesClassiqk:},\textbf{@CharlesClassiqk:},@CharlesClassiqk:,@CharlesClassiqk:
181,sorry,\textbf{sorry},\textbf{sorry},\textbf{sorry},sorry,sorry
182,\textbf{I'm},I'm,\textbf{I'm},\textbf{I'm},I'm,I'm
183,\textbf{not},not,\textbf{not},\textbf{not},not,not
184,\textbf{sexist},\textbf{sexist},\textbf{sexist},\textbf{sexist},sexist,sexist
185,BUT,BUT,BUT,BUT,BUT,\textbf{BUT}
186,there,\textbf{there},there,there,there,\textbf{there}
187,is,is,is,is,is,is
188,a,a,a,a,a,a
189,double,\textbf{double},double,\textbf{double},double,\textbf{double}


In [6]:
print(alldoms[180:202].to_latex(index=False, escape=False))

\begin{tabular}{llllll}
\toprule
            bl-fce &            Mean_fix_dur-fce &         bl-semeval_neg_sent & Mean_fix_dur-semeval_neg_sent &          bl-wh_both & Mean_fix_dur-wh_both \\
\midrule
 @CharlesClassiqk: &  \textbf{@CharlesClassiqk:} &  \textbf{@CharlesClassiqk:} &    \textbf{@CharlesClassiqk:} &   @CharlesClassiqk: &    @CharlesClassiqk: \\
             sorry &              \textbf{sorry} &              \textbf{sorry} &                \textbf{sorry} &               sorry &                sorry \\
      \textbf{I'm} &                         I'm &                \textbf{I'm} &                  \textbf{I'm} &                 I'm &                  I'm \\
      \textbf{not} &                         not &                \textbf{not} &                  \textbf{not} &                 not &                  not \\
   \textbf{sexist} &             \textbf{sexist} &             \textbf{sexist} &               \textbf{sexist} &              sexist &               sexist \\
    

# Waseem Hovy Hatespeech

In [403]:
def splitdataset(ssents):
    nsents = len(ssents)
    dv_startidx=int(nsents*0.8)
    te_start_idx=np.int(nsents*0.9)

    tr = ssents[:dv_startidx]
    dv = ssents[dv_startidx:te_start_idx]
    te = ssents[te_start_idx:]
    return tr, dv, te

def printtofile(ssents, outfilename):

    tr, dv, te = splitdataset(ssents)
    print(len(tr))
    print(len(te))
    print(len(dv))
    
    allsplit = [tr, dv, te]
    splitnames = ['tr', 'dv', 'te']
    for i, split in enumerate(allsplit):
        splitname = splitnames[i]
        outfile=codecs.open(outfilename+'.'+splitname+'.tsv', mode='w', encoding='utf-8')

        for sent in split:
            if len(sent) == 2:
                print(sent[0], file=outfile)
                words = sent[1]
                words = words.split()
                [print(word+'\t_', file=outfile) for word in words]
                print('\n', file=outfile)
        outfile.close() 

In [389]:
wh = pd.read_json('Data/hatespeech_data/waseem_hovy.json', lines=True, encoding='utf-8')
wh = wh[['Annotation', 'text']]
len(wh)

14031

In [390]:
#very skewed distribution
w = pd.read_json('Data/hatespeech_data/waseem.json', lines=True, encoding='utf-8')
w = w[['Annotation', 'text']]
len(w)

6909

In [391]:
#preprocess wh
wh.text = wh.text.str.replace('\n', '')
wh.text = wh.text.str.replace('\t', '')

sexism = wh.loc[(wh.Annotation == 'sexism') | (wh.Annotation == 'none')]
racism = wh.loc[(wh.Annotation == 'racism') | (wh.Annotation == 'none')]

sexism.Annotation.replace('sexism', '+', inplace=True)
sexism.Annotation.replace('none', '-', inplace=True)

racism.Annotation.replace('racism', '+', inplace=True)
racism.Annotation.replace('none', '-', inplace=True)

sexism.to_csv('Data/hatespeech_data/wh_sexism.tsv', sep='\t', encoding='utf-8', index=None, header=None)
racism.to_csv('Data/hatespeech_data/wh_racism.tsv', sep='\t', encoding='utf-8', index=None, header=None)

/Users/Maria/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [392]:
#preprocess w

w.text = w.text.str.replace('\n', '')
w.text = w.text.str.replace('\t', '')

sexism = w.loc[(w.Annotation == 'Sexism') | (w.Annotation == 'Neither') | (w.Annotation == 'Both') ]
racism = w.loc[(w.Annotation == 'Racism') | (w.Annotation == 'Neither') | (w.Annotation == 'Both')]

sexism.Annotation.replace('Sexism', '+', inplace=True)
sexism.Annotation.replace('Both', '+', inplace=True)
sexism.Annotation.replace('Neither', '-', inplace=True)

racism.Annotation.replace('Racism', '+', inplace=True)
racism.Annotation.replace('Both', '+', inplace=True)
racism.Annotation.replace('Neither', '-', inplace=True)

sexism.to_csv('Data/hatespeech_data/w_sexism.tsv', sep='\t', encoding='utf-8', index=None, header=None)
racism.to_csv('Data/hatespeech_data/w_racism.tsv', sep='\t', encoding='utf-8', index=None, header=None)

/Users/Maria/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [451]:
#preprocess wh and join sexism and racism in on category

both = wh.copy(deep=True)

both.Annotation.replace('sexism', '+', inplace=True)
both.Annotation.replace('racism', '+', inplace=True)
both.Annotation.replace('none', '-', inplace=True)

both.to_csv('Data/hatespeech_data/wh_both.tsv', sep='\t', encoding='utf-8', index=None, header=None)

bothw = w.copy(deep=True)

bothw.Annotation.replace('Sexism', '+', inplace=True)
bothw.Annotation.replace('Racism', '+', inplace=True)
bothw.Annotation.replace('Both', '+', inplace=True)
bothw.Annotation.replace('Neither', '-', inplace=True)

bothw.to_csv('Data/hatespeech_data/w_both.tsv', sep='\t', encoding='utf-8', index=None, header=None)

In [394]:
sexismf = codecs.open('Data/hatespeech_data/w_sexism.tsv', 'r', 'utf-8').read().split('\n')
w_ssents = [li.split('\t') for li in sexismf]
racismf = codecs.open('Data/hatespeech_data/w_racism.tsv', 'r', 'utf-8').read().split('\n')
w_rsents = [li.split('\t') for li in racismf]

printtofile(w_ssents , 'Data/hatespeech_data/w.sexism')
printtofile(w_rsents, 'Data/hatespeech_data/w.racism')

5449
682
681
4799
600
600


In [395]:
sexismf = codecs.open('Data/hatespeech_data/wh_sexism.tsv', 'r', 'utf-8').read().split('\n')
wh_ssents = [li.split('\t') for li in sexismf]
racismf = codecs.open('Data/hatespeech_data/wh_racism.tsv', 'r', 'utf-8').read().split('\n')
wh_rsents = [li.split('\t') for li in racismf]

printtofile(wh_ssents , 'Data/hatespeech_data/wh.sexism')
printtofile(wh_rsents, 'Data/hatespeech_data/wh.racism')

9646
1206
1206
8588
1074
1073


In [399]:
both = codecs.open('Data/hatespeech_data/wh_both.tsv', 'r', 'utf-8').read().split('\n')
both_sents = [li.split('\t') for li in both]
printtofile(both_sents , 'wh.both')
print(Counter([sent[0] for sent in both_sents]))

11225
1404
1403
Counter({'-': 8760, '+': 5271, '': 1})


In [456]:
wboth = codecs.open('Data/hatespeech_data/w_both.tsv', 'r', 'utf-8').read().split('\n')
wboth_sents = [li.split('\t') for li in wboth]
printtofile(wboth_sents , 'Data/hatespeech_data/w.both')
print(Counter([sent[0] for sent in wboth_sents]))

5528
691
691
Counter({'-': 5850, '+': 1059, '': 1})


# Dundee and ZuCo

In [77]:
dundee = pd.read_csv('Data/EN_dun.csv', encoding='utf-8', sep='\t', index_col=0)

In [78]:
#No frequency cols, no word length count.
dundeedatacols = ['First_fix_dur', 'First_pass_dur', 'Fix_prob', 'Fixation_prob', 'LINE',
       'Mean_fix_dur', 'OBLP', 'OLEN', 'Re-read_prob', 'TXFR',
       'Tot_fix_dur', 'Tot_regres_from_dur', 'Tot_regres_to_dur', 'WDLP',
       'nFix', 'nLong_regres_from', 'nLong_regres_to',
       'nRefix', 'nRefixa_mean', 'nRegres_from', 'nRegres_to',
       'nRegressions_to', 'n-1_fix_prob', 'n+1_fix_prob', 'n-2_fix_prob',
       'n+2_fix_prob', 'n-1_fix_dur', 'n+1_fix_dur', 'n-2_fix_dur',
       'n+2_fix_dur', 'RelWDLP', 'RelOBLP', 'LAUN', 'LAUNabs']

nora_datacols = ['nFixations','FFD', 'TRT', 'GD', 'SFD']
usecols = ['sent-id', 'word-id', 'word']
usecols.extend(nora_datacols)

joint_datacols = ['First_fix_dur', 'First_pass_dur', 'Mean_fix_dur','Tot_fix_dur', 'nFix', 'BNCfreq', 'BNCfreqinv','WordLen'  ]
word_and_joint = ['Word', 'First_fix_dur', 'First_pass_dur', 'Mean_fix_dur','Tot_fix_dur', 'nFix', 'BNCfreq', 'BNCfreqinv', 'WordLen' ]

In [79]:
len(dundee.groupby(['Itemno', 'SentenceID']))

2368

In [80]:
len(dundee) / len(dundee.groupby(['Itemno', 'SentenceID']))

24.746199324324323

In [81]:
unigrfile = codecs.open('Data/BNC_all.al', 'r',encoding='utf-8').read().split('\n')
unilist = [li.split() for li in unigrfile]
unigrdict = {}
for li in unilist[1:]:
    if len(li) == 4:
        try:
            unigrdict[li[1]]+=float(li[0])
        except KeyError:
            unigrdict[li[1]]=float(li[0])

In [82]:
#Reading in datafiles from NR and SR, grouping by word and rename columns according to dundee col names
#concat to zurich

allNR = pd.DataFrame(columns=usecols)

for fname in glob.glob('Data/word-level-data/data_NR/*.csv'):
    onefile = pd.read_csv(fname, index_col=None, encoding='utf-8')
    onefile = onefile[usecols]
    onefile.replace('[]', 0, inplace=True)
    allNR = pd.concat([allNR, onefile])

allNR.loc[:,nora_datacols] = allNR[nora_datacols].astype('float')

allNR['Mean_fix_dur'] = allNR.TRT.astype(int) / allNR.nFixations.astype(int) #int division by zero returns nan
allNR.Mean_fix_dur.fillna(0, inplace=True) #replace nans with 0

allNR[nora_datacols] = allNR[nora_datacols].astype('float')
allNRgr = allNR.groupby(['sent-id', 'word-id', 'word'], as_index=False).mean()
allNRgr.rename(columns={'word':'Word','FFD':'First_fix_dur', 'TRT':'Tot_fix_dur', 'nFixations':'nFix', 'GD':'First_pass_dur'}, inplace=True)
allNRgr['sent-id'] = allNRgr['sent-id'].apply(lambda x: str(x)+'_NR')

allSR = pd.DataFrame(columns=usecols)

for fname in glob.glob('Data/word-level-data/data_SR/*.csv'):
    onefile = pd.read_csv(fname, index_col=None, encoding='utf-8')
    onefile = onefile[usecols]
    onefile.replace('[]', 0, inplace=True)
    allSR = pd.concat([allSR, onefile])
    
allSR['Mean_fix_dur'] = allSR.TRT.astype(int) / allSR.nFixations.astype(int) #int division by zero returns nan
allSR.Mean_fix_dur.fillna(0, inplace=True)

allSR.loc[:,nora_datacols] = allSR[nora_datacols].astype('float')

allSR['Mean_fix_dur'] = allSR.TRT.divide(allSR.nFixations)
allSR.Mean_fix_dur.fillna(0, inplace=True) #replace nans with 0

allSRgr = allSR.groupby(['sent-id', 'word-id', 'word'], as_index=False).mean()
allSRgr.rename(columns={'word':'Word','FFD':'First_fix_dur', 'TRT':'Tot_fix_dur', 'nFixations':'nFix', 'GD':'First_pass_dur'}, inplace=True)
allSRgr['sent-id'] = allSRgr['sent-id'].apply(lambda x: str(x)+'_SR')


zurich = pd.concat([allSRgr, allNRgr])
#zurich = allSRgr #======= only using Sentimentreading! ==========
zurich['WORDstrip'] = zurich['Word'].astype('str')
zurich.WORDstrip = zurich.WORDstrip.str.replace('[^\w\s]','')
zurich['WordLen'] = zurich.WORDstrip.str.len()

zurich['BNCfreq'] = zurich.WORDstrip.map(lambda x: unigrdict.get(str(x).lower()))
zurich.BNCfreq = zurich.BNCfreq.fillna(zurich.BNCfreq.min())
zurich.BNCfreq = zurich.BNCfreq/100 #because 100 million word - to get freq per million
zurich.BNCfreq = np.log(zurich.BNCfreq)

dundee['BNCfreqcount'] = dundee.Word.map(lambda x: unigrdict.get(str(x).lower()))
dundee.BNCfreqcount = dundee.BNCfreqcount.fillna(dundee.BNCfreqcount.min())
dundee['BNCfreq'] = dundee.BNCfreqcount/100  
dundee.BNCfreq = np.log(dundee.BNCfreq)

dundee['BNCfreqinv'] = -dundee.BNCfreq
zurich['BNCfreqinv']= -zurich.BNCfreq

In [65]:
"""len(dundee)
dundee['tot_fix_dur_ms'] = np.round(dundee.Tot_fix_dur *1000, 0)

print(dundee[['Word', 'UniversalPOS', 'tot_fix_dur_ms']][1240:1252].T.to_latex())"""

\begin{tabular}{lllllllllllll}
\toprule
{} &         1240 &     1241 &  1242 & 1243 &      1244 &     1245 & 1246 & 1247 &      1248 &  1249 &  1250 &         1251 \\
\midrule
Word           &  Responsible &  tourism &  lies &   in &  absolute &  respect &  for &  the &  location &   and &   its &  inhabitants \\
UniversalPOS   &          ADJ &     NOUN &  VERB &  ADP &       ADJ &     NOUN &  ADP &  DET &      NOUN &  CONJ &  PRON &         NOUN \\
tot\_fix\_dur\_ms &          319 &      271 &   319 &  169 &       255 &      208 &  162 &  191 &       205 &   108 &   147 &          382 \\
\bottomrule
\end{tabular}



In [50]:
"""#No wlen bins and no freq bins
def get_bin_stdfrommean(dundee):
    for col in joint_datacols:
        dundee[col+'_mean'] = dundee[col].mean()
        dundee[col+'_std'] = dundee[col].std()
        dundee[col+'_stdfrommean'] = (dundee[col] - dundee[col+'_mean']) / dundee[col+'_std']
        dundee[col+'_stdfrommeanbin'] = 0
        dundee.loc[dundee[col+'_stdfrommean'] >= 1, col+'_stdfrommeanbin'] = 1
        dundee.drop([col+'_stdfrommean', col+'_std', col+'_mean'], axis=1, inplace=True)
    return dundee

#only BNC freq
def get_bin_stdfrommean_freq(dundee):
    col='BNCfreq'
    dundee[col+'_mean'] = dundee[col].mean()
    dundee[col+'_std'] = dundee[col].std()
    dundee[col+'_stdfrommean'] = (dundee[col] - dundee[col+'_mean']) / dundee[col+'_std']
    dundee[col+'_stdfrommeanbin'] = 0
    dundee.loc[dundee[col+'_stdfrommean'] <= -1, col+'_stdfrommeanbin'] = 1 #smaller than - pay attention to infrequent words
    dundee.drop([col+'_stdfrommean', col+'_std', col+'_mean'], axis=1, inplace=True)
    return dundee

dundee = get_bin_stdfrommean(dundee)
zurich = get_bin_stdfrommean(zurich)
dundee = get_bin_stdfrommean_freq(dundee)
zurich = get_bin_stdfrommean_freq(zurich)"""

In [30]:
"""#defining bins - taking min and max values from dundee and zurich as endpoints
#making bins based on values from both geco and dundee
bins= 7
freqbins = pd.cut(pd.concat([zurich.BNCfreq, dundee.BNCfreq]), bins, retbins=True)
dundee['Freqgroup'] = pd.cut(dundee.BNCfreq, bins = freqbins[1], labels = [i for i in range(bins)])
zurich['Freqgroup'] = pd.cut(zurich.BNCfreq, bins = freqbins[1], labels = [i for i in range(bins)])

wlenbins = pd.cut(pd.concat([zurich.Wordlen, dundee.WordLen]), bins, retbins=True)
zurich['Wlengroup'] = pd.cut(zurich.Wordlen, bins = wlenbins[1], labels=[i for i in range(bins)])
dundee['Wlengroup'] = pd.cut(dundee.WordLen, bins = wlenbins[1], labels = [i for i in range(bins)])"""

In [564]:
"""stdfrommean = 1 #positive label is >= stdfrommean
#taking the mean and std for each measure for each group
for colname in dundeedatacols:
    groupbyitem = dundee.groupby(['Wlengroup', 'Freqgroup'])[colname]
    dundee = dundee.join(groupbyitem.mean(), rsuffix='_meangroup', on=['Wlengroup', 'Freqgroup'])
    dundee = dundee.join(groupbyitem.std(), rsuffix='_stdgroup', on=['Wlengroup', 'Freqgroup'])
    dundee[colname+'_stdgroup'].fillna(1, inplace=True) #for the very few groups with only 1 word. They have no std.
    dundee[colname+'_stdfrommean'] = (dundee[colname] - dundee[colname+'_meangroup']) / dundee[colname+'_stdgroup']
    dundee.drop(colname+'_meangroup', axis=1, inplace=True)
    dundee.drop(colname+'_stdgroup', axis=1, inplace=True)
    #binarize _stdfrommean-cols
    dundee.loc[dundee[colname+'_stdfrommean'] >= stdfrommean, colname+'_stdfrommean' ] = 1
    dundee.loc[dundee[colname+'_stdfrommean'] < stdfrommean, colname+'_stdfrommean'] = 0
    dundee[colname+'_stdfrommean'].replace({0: '-', 1: '+'}, inplace=True)
    
for colname in joint_datacols:
    groupbyitem = zurich.groupby(['Wlengroup', 'Freqgroup'])[colname]
    zurich = zurich.join(groupbyitem.mean(), rsuffix='_meangroup', on=['Wlengroup', 'Freqgroup'])
    zurich = zurich.join(groupbyitem.std(), rsuffix='_stdgroup', on=['Wlengroup', 'Freqgroup'])
    zurich[colname+'_stdgroup'].fillna(1, inplace=True) #for the very few groups with only 1 word. They have no std.
    zurich[colname+'_stdfrommean'] = (zurich[colname] - zurich[colname+'_meangroup']) / zurich[colname+'_stdgroup']
    zurich.drop(colname+'_meangroup', axis=1, inplace=True)
    zurich.drop(colname+'_stdgroup', axis=1, inplace=True)
    #binarize _stdfrommean-cols
    zurich.loc[zurich[colname+'_stdfrommean'] >= stdfrommean, colname+'_stdfrommean' ] = 1
    zurich.loc[zurich[colname+'_stdfrommean'] < stdfrommean, colname+'_stdfrommean'] = 0
    zurich[colname+'_stdfrommean'].fillna(0, inplace=True) #First pass duration missing values - resulting in nans
    zurich[colname+'_stdfrommean'].replace({0: '-', 1: '+'}, inplace=True)"""

KeyError: 'Wlengroup'

In [83]:
#Use continous values min -max normalize
#zurich.Mean_fix_dur.fillna(0, inplace=True) #could probably be commented out

zurich[joint_datacols] = (zurich[joint_datacols]-zurich[joint_datacols].min())/(zurich[joint_datacols].max()-zurich[joint_datacols].min())
dundee[joint_datacols] = (dundee[joint_datacols]-dundee[joint_datacols].min())/(dundee[joint_datacols].max()-dundee[joint_datacols].min())

In [91]:
zurich.

,sent-id,word-id,Word,nFix,First_fix_dur,Tot_fix_dur,First_pass_dur,SFD,Mean_fix_dur,WORDstrip,WordLen,BNCfreq,BNCfreqinv
0,0_SR,0,Presents,0.156863,0.238623,0.124402,0.123658,42.416667,0.264039,Presents,0.32,0.499857,0.500143
1,0_SR,1,a,0.053922,0.111554,0.029020,0.048249,50.833333,0.116627,a,0.04,0.932427,0.067573
2,0_SR,2,good,0.151961,0.171838,0.098537,0.087935,19.750000,0.189884,good,0.16,0.722573,0.277427
3,0_SR,3,case,0.102941,0.186761,0.078655,0.115350,38.166667,0.208777,case,0.16,0.682662,0.317338
4,0_SR,4,while,0.147059,0.215278,0.096687,0.120782,17.166667,0.206363,while,0.20,0.699861,0.300139
5,0_SR,5,failing,0.181373,0.184840,0.126251,0.128195,21.916667,0.196909,failing,0.28,0.494119,0.505881
6,0_SR,6,to,0.039216,0.088948,0.022710,0.038471,33.916667,0.091755,to,0.08,0.943570,0.056430
7,0_SR,7,provide,0.166667,0.256206,0.112625,0.124233,22.750000,0.222849,provide,0.28,0.640313,0.359687
8,0_SR,8,a,0.029412,0.055851,0.017733,0.024156,24.666667,0.061269,a,0.04,0.932427,0.067573
9,0_SR,9,reason,0.171569,0.221188,0.111075,0.151074,11.500000,0.208575,reason,0.24,0.628026,0.371974


In [90]:
len(zurich.groupby('WORDstrip'))

4651

In [24]:
#insert blank line after each sentence - for GECO we don't have sentence boundaries
zurichblank = pd.Series([np.nan]*len(zurich.columns), index=zurich.columns)
zurich = zurich.groupby(['sent-id'], as_index=False).apply(lambda x: x.append(zurichblank, ignore_index=True)).reset_index(drop=True)

dundeeblank = pd.Series([np.nan]*len(dundee.columns), index=dundee.columns)
dundee = dundee.groupby(['Itemno', 'SentenceID'], as_index=False).apply(lambda x: x.append(dundeeblank, ignore_index=True)).reset_index(drop=True)

#concatenate dundee and zurich

both = pd.concat([zurich[word_and_joint], dundee[word_and_joint]]).reset_index(drop=True)

In [25]:
#for colname in joint_datacols:
#    dundee[['Word', colname+'_stdfrommean']].to_csv('exp_data/dundee'+colname+'.csv', sep='\t', encoding='utf-8', index=None, header=None)
     
#for colname in joint_datacols:
#    zurich[['Word', colname]].to_csv('exp_data/zurich_nogroup_SR'+colname+'.csv', sep='\t', encoding='utf-8', index=None, header=None)
    
for colname in joint_datacols:
    both[['Word', colname]].to_csv('exp_data/both_cont_'+colname+'.csv', sep='\t', encoding='utf-8', index=None, header=None)

In [574]:
def read_input_files(file_paths, max_sentence_length=-1):
    """
    Reads input files in whitespace-separated format.
    Will split file_paths on comma, reading from multiple files.
    """
    # JB: I've changed this so a sentence-level label can be read at the
    # beginning of the sentence (single label on extra line preceding sentence)
    # output is a list of (sentence, label) tuples, where sentence is a list of
    # (token, token_label) tuples
    sentences = []
    line_length = None
    sent_label = ""
    for file_path in file_paths.strip().split(","):
        with open(file_path, "r") as f:
            ttsentence = []
            for line in f:
                line = line.strip()
                if len(line) == 1:
                    sent_label = line
                elif len(line) > 1:
                    line_parts = line.split()
                    assert(len(line_parts) >= 2), line
                    assert(len(line_parts) == line_length or line_length == None)
                    line_length = len(line_parts)
                    sentence.append(line_parts)
                elif len(line) == 0 and len(sentence) > 0:
                    if max_sentence_length <= 0 or len(sentence) <= max_sentence_length:
                        sentences.append((sentence, sent_label))
                    sentence = []
                    sent_label = ""
            if len(sentence) > 0:
                if max_sentence_length <= 0 or len(sentence) <= max_sentence_length:
                    sentences.append((sentence, sent_label))
    return sentences

In [575]:
a = read_input_files('exp_data/both_cont_Mean_fix_dur.csv')

# Semeval2014

In [220]:
f = codecs.open('Data/SemEval2014-Task9-subtaskAB-test-to-download/SemEval2014-task9-test-B-gold.txt', 'r', 'utf-8').read().split('\n')
sents = [li.split('\t') for li in f]
posfile=codecs.open("Data/SemEval2014-Task9-subtaskAB-test-to-download/Semeval2014.test.pos", mode='w', encoding='utf-8')
negfile =codecs.open("Data/SemEval2014-Task9-subtaskAB-test-to-download/Semeval14.test.neg", mode='w', encoding='utf-8')

for sent in sents:
    if len(sent)==4 and sent[3] != 'Not Available':
        if sent[2] == 'neutral':
            print('-', file=posfile)
            print('-', file=negfile)
            words = sent[3]
            words = words.split()
            [print(word+'\t_', file=negfile) for word in words]
            [print(word+'\t_', file=posfile) for word in words]
            print('\n', file=posfile)
            print('\n', file=negfile)
            
        elif sent[2] == 'positive':
            print('+', file=posfile)
            words = sent[3]
            words = words.split()
            [print(word+'\t_', file=posfile) for word in words]
            print('\n', file=posfile)
        elif sent[2] == 'negative':
            print('+', file=negfile)
            [print(word+'\t_', file=negfile) for word in words]
            print('\n', file=negfile)
posfile.close()
negfile.close()

In [223]:
f = codecs.open('Data/SemEval2014-Task9-subtaskAB-test-to-download/SemEval2014-task9-dev-B-gold.txt', 'r', 'utf-8').read().split('\n')
sents = [li.split('\t') for li in f]
posfile=codecs.open("Data/SemEval2014-Task9-subtaskAB-test-to-download/Semeval2014.dev.pos", mode='w', encoding='utf-8')
negfile =codecs.open("Data/SemEval2014-Task9-subtaskAB-test-to-download/Semeval14.dev.neg", mode='w', encoding='utf-8')

for sent in sents:
    if len(sent)==4 and sent[3] != 'Not Available':
        if sent[2] == 'neutral':
            print('-', file=posfile)
            print('-', file=negfile)
            words = sent[3]
            words = words.split()
            [print(word+'\t_', file=negfile) for word in words]
            [print(word+'\t_', file=posfile) for word in words]
            print('\n', file=posfile)
            print('\n', file=negfile)
            
        elif sent[2] == 'positive':
            print('+', file=posfile)
            words = sent[3]
            words = words.split()
            [print(word+'\t_', file=posfile) for word in words]
            print('\n', file=posfile)
        elif sent[2] == 'negative':
            print('+', file=negfile)
            [print(word+'\t_', file=negfile) for word in words]
            print('\n', file=negfile)
posfile.close()
negfile.close()

# Amazon reviews

In [224]:
# As one task? Just test set? Is binary task - no neutral reviews

In [13]:
from bs4 import BeautifulSoup
import random
import re
from nltk.tokenize import sent_tokenize

In [14]:
def parse_reviews(pathtofiles):
    random.seed(99)
    labs = ['+', '-']
    reviews = []
    for i, fname in enumerate(['positive.review', 'negative.review']):
        soup = BeautifulSoup(open(pathtofiles+fname), 'xml')
        for review in soup.find_all('review'):
            reviewsents = sent_tokenize(review.review_text.text.strip('\n'))
            for sent in reviewsents:
                temp = []
                temp.append(labs[i])

                temp.append(sent)

                reviews.append(temp)
    random.shuffle(reviews)
    
    dev_size = 0.1
    tr_size = 0.7
    te_size = 0.2
    
    lenghtofreview = len(reviews)

    tr_startidx=int(lenghtofreview*dev_size)
    te_start_idx=np.int(lenghtofreview*(dev_size+tr_size))
    
    dv_reviews = reviews[:tr_startidx]
    tr_reviews = reviews[tr_startidx:te_start_idx]
    te_reviews = reviews[te_start_idx:]
    
    return dv_reviews, tr_reviews, te_reviews

In [15]:
kitchen_dv, kitchen_tr, kitchen_te = parse_reviews('Data/Amazon_sorted_data/kitchen_&_housewares/')
electronics_dv, electronics_tr, electronics_te = parse_reviews('Data/Amazon_sorted_data/electronics/')
books_dv, books_tr, books_te = parse_reviews('Data/Amazon_sorted_data/books/')
dvd_dv, dvd_tr, dvd_te = parse_reviews('Data/Amazon_sorted_data/dvd/')

In [19]:
print(len(kitchen_te))
print(len(books_te))
print(len(dvd_te))
print(len(electronics_te))


2340
3480
3813
2661


In [370]:
fnames = ['kitchen_dv', 'kitchen_tr', 'kitchen_te', 'electronics_dv', 'electronics_tr', 'electronics_te', 
          'books_dv', 'books_tr', 'books_te','dvd_dv', 'dvd_tr', 'dvd_te']


for i, domain in enumerate([kitchen_dv, kitchen_tr, kitchen_te, electronics_dv, electronics_tr, electronics_te, 
          books_dv, books_tr, books_te, dvd_dv, dvd_tr, dvd_te]):
    outfile = codecs.open('Data/Amazon_sorted_data/'+fnames[i]+'.txt', 'w', encoding='utf-8')
    for sent in domain:
        print(sent[0], file=outfile)
        words = sent[1].split()
        [print(word+'\t_', file=outfile) for word in words]
        print('\n', file=outfile)

# Geco

In [18]:
geco = pd.read_csv('Data/GECOmonoling.csv', encoding='utf-8', sep='\t')

gecodatacols = ['WORD_AVERAGE_FIX_PUPIL_SIZE',
       'WORD_FIXATION_COUNT', 'WORD_RUN_COUNT',
       'WORD_FIRST_RUN_FIXATION_COUNT',
       'WORD_GAZE_DURATION', 'WORD_SECOND_RUN_START_TIME',
       'WORD_SECOND_RUN_END_TIME', 'WORD_SECOND_RUN_FIXATION_COUNT',
        'WORD_THIRD_RUN_FIXATION_COUNT', 'WORD_FIRST_FIXATION_DURATION',
       'WORD_FIRST_FIXATION_TIME', 'WORD_FIRST_FIXATION_VISITED_WORD_COUNT',
       'WORD_FIRST_FIX_PROGRESSIVE', 'WORD_SECOND_FIXATION_DURATION',
       'WORD_SECOND_FIXATION_RUN', 'WORD_SECOND_FIXATION_TIME',
       'WORD_THIRD_FIXATION_DURATION', 'WORD_THIRD_FIXATION_RUN',
       'WORD_THIRD_FIXATION_TIME', 'WORD_LAST_FIXATION_DURATION',
       'WORD_LAST_FIXATION_RUN', 'WORD_LAST_FIXATION_TIME', 'WORD_GO_PAST_TIME',
       'WORD_SELECTIVE_GO_PAST_TIME', 'WORD_TOTAL_READING_TIME', 'WORD_SPILLOVER', 'WORD_SKIP',
       'fix_prob']

for colname in gecodatacols:
    geco[colname] = geco[colname].astype(float)
    groupbyitem = geco.groupby(['Wlengroup', 'Freqgroup'])[colname]
    geco = geco.join(groupbyitem.mean(), rsuffix='_meangroup', on=['Wlengroup', 'Freqgroup'])
    geco = geco.join(groupbyitem.std(), rsuffix='_stdgroup', on=['Wlengroup', 'Freqgroup'])
    geco[colname+'_stdgroup'].fillna(1, inplace=True) #for the very few groups with only 1 word. They have no std.
    geco[colname+'_stdfrommean'] = (geco[colname] - geco[colname+'_meangroup']) / geco[colname+'_stdgroup']
    geco.drop(colname+'_meangroup', axis=1, inplace=True)
    geco.drop(colname+'_stdgroup', axis=1, inplace=True)
    #binarize _stdfrommean-cols
    geco.loc[geco[colname+'_stdfrommean'] >= 0, colname+'_stdfrommean'] = 1
    geco.loc[geco[colname+'_stdfrommean'] < 0, colname+'_stdfrommean'] = 0
    geco[colname+'_stdfrommean'].replace({0: '-', 1: '+'}, inplace=True)
    
geco['WORDstrip'] = geco['WORD'].astype('str')
geco.WORDstrip = geco.WORDstrip.str.replace('[^\w\s]','')
geco['Wordlen'] = geco.WORDstrip.str.len()
geco['BNCfreq'] = geco.WORDstrip.map(lambda x: unigrdict.get(x.lower()))
geco.BNCfreq = geco.BNCfreq.fillna(geco.BNCfreq.min())
geco.BNCfreq = geco.BNCfreq/1000000
geco.BNCfreq = np.log(geco.BNCfreq)


KeyError: 'Wlengroup'

# change c / i to + and - in fce

In [ ]:
for fname in glob.glob('Data/test/fce-error-detection/tsv/*original.tsv',encoding='utf-8' ):
    df = pd.read_csv(fname, header=None, names=['word', 'label'], index_col=None, sep='\s+', quoting=csv.QUOTE_NONE, skip_blank_lines=False)
    df['label'].replace('c', '-', inplace=True)
    df['label'].replace('i', '+', inplace=True)
    df.to_csv(fname, header=None, index=None, encoding='utf-8', sep='\t')

# make sentence-level labels for FCE

In [425]:
def sentlabel(foldername, ext):
    #all files with ext in foldername are given sentence label + if there is one + in the tokenlevel labels. 
    #Otherwise -
    for fname in glob.glob(foldername+'*'+'.'+ext):
        f = open(fname, mode='r', encoding='utf-8').read().split('\n\t')
        outfile = open(fname[:-4]+'sentlab'+'.'+ext, mode='w', encoding='utf-8')
        content = [[l.split('\t') for l in li.split('\n')] for li in f]
        for sent in content:
            toklabs = [li[1] for li in sent if len(li) > 1]
            if '+' in set(toklabs):
                print("+", file=outfile)     
            else:
                print("-", file=outfile)
            [print(li[0]+'\t_', file=outfile) for li in sent if len(li) ==2]
            print("\n", file=outfile, end='')
        outfile.close()

sentlabel('Data/fce-error-detection/tsv/', 'tsv')

# Sentiment semeval labels

In [ ]:
# Convert 0 and P / N to + and - for tokenfiles
for fname in glob.glob('Data/semeval15t10/semeval15t10.tokens*.tsv'):
    df = pd.read_csv(fname, header=None, names=['word', 'label'], index_col=None, sep='\s+', quoting=csv.QUOTE_NONE, skip_blank_lines=False)
    df['label'].replace('O', '-', inplace=True)
    df['label'].replace('P', '+', inplace=True)
    df['label'].replace('N', '+', inplace=True)
    df.to_csv(fname, header=None, index=None, encoding='utf-8', sep='\t')

In [64]:
#get sentence level label from tokens and put in beginning of line
#sent-annotated sentences are not aligned with token-level annotated, so we can't use sentlabel on token

for fname in glob.glob('Data/test2013twitter/semeval15t10.sentences.*.test2013twitter.tsv'):
    print(fname)
    outfile = open(fname[:-4]+'sentlab'+'.tsv', mode='w', encoding='utf-8')
    sentfile = open(fname, mode='r', encoding='utf-8').read().split('\n\n')
    sentcontent = [[l.split('\t') for l in li.split('\n')] for li in sentfile]
    
    for sent in sentcontent:
        sentlabs = [li[1] for li in sent if len(li) > 1]
        if 'O' not in set(sentlabs) and len(set(sentlabs)) > 0: #sent use O, P and N
            print("+", file=outfile)     
        else:
            print("-", file=outfile)
        [print(li[0]+'\t'+'_', file=outfile) for li in sent if len(li) ==2] # blank token label
        print("\n", file=outfile, end='')
outfile.close()

Data/test2013twitter/semeval15t10.sentences.negative.test2013twitter.tsv
Data/test2013twitter/semeval15t10.sentences.positive.test2013twitter.tsv


# Draft

In [523]:
resfile = 'mltagger/exp/fce.dundee.First_fix_dur/output.txt'
task = 'fce'
split='dev'

[['cost_sum', '9950.793182373047'],
 ['cost_avg', '3.855402240361506'],
 ['sent_count', '2581.0'],
 ['sent_predicted', '1629.0'],
 ['sent_correct', '1069.0'],
 ['sent_total', '1571.0'],
 ['sent_p', '0.6562308164518109'],
 ['sent_r', '0.6804583068109484'],
 ['sent_f', '0.668125'],
 ['sent_f05', '0.6609373067886731'],
 ['sent_correct_binary', '1519.0'],
 ['sent_accuracy_binary', '0.5885315769081751'],
 ['tok_0_map', '0.32840218880676103'],
 ['tok_0_p', '0.1502044863056141'],
 ['tok_0_r', '0.4680440239428461'],
 ['tok_0_f', '0.22742412159309472'],
 ['tok_0_f05', '0.17381078716783066'],
 ['time', '28.54233431816101'],
 ['best_epoch', '1']]

{'best_epoch': 1.0,
 'cost_avg': 3.855402240361506,
 'cost_sum': 9950.793182373047,
 'sent_': 0.668125,
 'sent_accuracy_binary': 0.5885315769081751,
 'sent_correct': 1069.0,
 'sent_correct_binary': 1519.0,
 'sent_count': 2581.0,
 'sent_f05': 0.6609373067886731,
 'sent_p': 0.6562308164518109,
 'sent_predicted': 1629.0,
 'sent_r': 0.6804583068109484,
 'sent_total': 1571.0,
 'tim': 28.54233431816101,
 'tok_0_': 0.22742412159309472,
 'tok_0_f05': 0.17381078716783066,
 'tok_0_map': 0.32840218880676103,
 'tok_0_p': 0.1502044863056141,
 'tok_0_r': 0.4680440239428461}

# generate synthetic data

In [76]:
import random
from  builtins import any as b_any

for fname in ['tr', 'dv', 'te']:
    outfile = open('Data/toy/'+fname+'.tsv', mode='w', encoding='utf-8')
    for i in range(50):
        wordsents = []
        for j in range(random.randint(5,10)):
            wordsents.append(''.join(random.choices(['a']*15+['A'], k=random.randint(2,5))))
        if b_any('A' in x for x in wordsents):
            print('+', file=outfile)
        else:
            print('-', file=outfile)
        [print(word+'\t'+'_',  file=outfile) for word in wordsents]
        print('\n', file=outfile, end='')
    outfile.close()

In [79]:
for fname in ['gazetr']:
    outfile = open('Data/toy/'+fname+'.tsv', mode='w', encoding='utf-8')
    for i in range(50):
        wordsents = []
        for j in range(random.randint(5,10)):
            wordsents.append(''.join(random.choices(['a']*15+['A'], k=random.randint(2,5))))
        for word in wordsents:
            if 'A' in word:
                print(word+'\t'+'+', file=outfile)
            elif 'A' not in word:
                print(word+'\t'+'-', file=outfile)
        print('\n', file=outfile, end='')
    outfile.close()

In [ ]:
frdundee = pd.read_csv()